In [1]:
import os
os.chdir('../') # insures use of models folder as python module 
# Any figures saved or data pickled in the current working 
# directly will be saved up one folder from here.
#import pybamm
#import models
import numpy as np
#import pickle
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['font.size'] =  18.0

In [700]:
class model:
    def __init__(self):
        # parameters

        # standard parameters
        self.R = 8.3145
        self.F = 9.649e4
        self.T = 303.15

        # parameters
        self.Ms = 32
        self.ns = 1
        self.ns2 = 2
        self.ns4 = 4
        self.ns8 = 8
        self.ne = 1
        self.ih0 = 5
        self.im0 = 5
        self.il0 = 5
        self.rho_s = 2000
        self.EH0 = 2.43
        self.EM0 = 2.41
        self.EL0 = 1.9
        self.v = 0.0114
        self.ar = 0.960
        self.k_p = 1
        self.k_d = 5000
        self.S_star = 1e-6
        self.k_s_charge = 1e-4
        self.k_s_discharge = 0
        
        
        
        # parameters derived from other parameters
        self.nH = 1
        self.nM = 1
        self.nL = 1
        self.iH_coef = self.nH * self.F / (2 * self.R * self.T)
        self.iM_coef = self.nM * self.F / (2 * self.R * self.T)
        self.iL_coef = self.nL * self.F / (2 * self.R * self.T)
        self.E_H_coef = self.R * self.T / (self.nH * self.F)
        self.E_M_coef = self.R * self.T / (self.nM * self.F)
        self.E_L_coef = self.R * self.T / (self.nL * self.F)
        self.f_h = ((self.ns4 ** 2) * self.Ms * self.v / self.ns8)**(1/4)
        self.f_m = ((self.ns2 ** 2) * self.Ms * self.v / self.ns4)**(1/2)
        self.f_l = ((self.ns ** 2) * self.Ms * self.v / self.ns2)**(1/2)

        #######################################################
        # Non-dynamic model functions
        #######################################################
        
        # standard Initial conditions 
        self.t = 0
        self.Sp = 0.43
        self.S1 = 0.001
        self.S2 = 0.011487966377185257
        self.S4 = 8.425161409668764e-18
        self.S8 = 2.467917664668259e-63
        self.V = 2.0
        self.I = -0.0211
        
        self.state = np.array([self.t, self.V, self.S8, self.S4, self.S2, self.S1, self.Sp])
        self.perma_state = self.state 
        
        self.t_data = [self.t]
        self.Sp_data = [self.Sp]
        self.S1_data = [self.S1]
        self.S2_data = [self.S2]
        self.S4_data = [self.S4]
        self.S8_data = [self.S8]
        self.V_data = [self.V]
        
    def E_H(self):
        S8 = self.state[2]
        S4 = self.state[3]
        return self.EH0 + self.E_H_coef * (np.log(self.f_h) + 0.25*np.log(S8) - 0.5*np.log(S4) )
    
    def E_M(self):
        S4 = self.state[3]
        S2 = self.state[4]
        return self.EM0 + self.E_M_coef * (np.log(self.f_m) + 0.5*np.log(S4) - np.log(S2) )
    
    def E_L(self):
        S2 = self.state[4]
        S1 = self.state[5]
        # Low plateau potenital [V] as defined by equation (2b) in [1]
        return self.EL0 + self.E_L_coef * (np.log(self.f_l) + 0.5*np.log(S2) - np.log(S1) ) 

        # High plateau over-potenital [V] as defined by equation (6a) in [1]
    def eta_H(self):
        V = self.state[1]
        return  V - self.E_H()
    
    def eta_M(self):
        V = self.state[1]
        return  V - self.E_M()
    
    def eta_L(self):
        V = self.state[1]
        return  V - self.E_L()

    def i_H(self):
        
        # High plateau current [A] as defined by equation (5a) in [1]
        return -2 * self.ih0 * self.ar * np.sinh(self.iH_coef * self.eta_H())
    
    def i_M(self):
        
        return -2 * self.im0 * self.ar * np.sinh(self.iM_coef * self.eta_M())
    
    def i_L(self):
        # Low plateau current [A] as defined by equation (5b) in [1]
        return -2 * self.il0 * self.ar * np.sinh(self.iL_coef * self.eta_L())
    
    def k_s(self):
        
        if self.I < 0:
            return self.k_s_charge
        else:
            return self.k_s_discharge
    def k_pd(self):
        if self.I < 0:
            return self.k_d
        else:
            return self.k_p
        
    def dS8dt(self):
        S8 = self.state[2]
        return -(self.ns8 * self.Ms * self.i_H() / (self.nH * self.F)) - self.k_s() * S8

    def dS4dt(self):
        S8 = self.state[2]
        return  (self.ns8 * self.Ms * self.i_H() / (self.nH * self.F)) + self.k_s() * S8 - (self.ns4 * self.Ms * self.i_M() / (self.nM * self.F))

    def dS2dt(self):
        return self.ns4 * self.Ms * self.i_M() / (self.nM * self.F) - (self.ns2 * self.Ms * self.i_L() / (self.nM * self.F) )

    def dS1dt(self):
        S1 = self.state[5]
        Sp = self.state[6]
        return (self.ns2 * self.Ms * self.i_L()  / (self.nM * self.F)) - self.k_pd() * Sp * (S1 - self.S_star) / (self.v * self.rho_s)
    
    def dSpdt(self):
        S1 = self.state[5]
        Sp = self.state[6]
        return self.k_pd() * Sp * (S1 - self.S_star) / (self.v * self.rho_s)
    
    def dVdt(self):
        S8 = self.state[2]
        S4 = self.state[3]
        S2 = self.state[4]
        S1 = self.state[5]
        numerator_H = self.ih0*np.cosh(self.iH_coef * self.eta_H())*( 0.25*(self.dS8dt()/S8) - 0.5*(self.dS4dt()/S4 ))
        numerator_M = self.im0*np.cosh(self.iM_coef * self.eta_M())*( 0.5*(self.dS4dt()/S4) - (self.dS2dt()/S2 ))
        numerator_L = self.il0*np.cosh(self.iL_coef * self.eta_L())*( 0.5*(self.dS2dt()/S2) - (self.dS1dt()/S1) )
        
        denominator_H = self.nH*self.ih0*np.cosh(self.iH_coef * self.eta_H())
        denominator_M = self.nM*self.im0*np.cosh(self.iM_coef * self.eta_M())
        denominator_L = self.nL*self.il0*np.cosh(self.iL_coef * self.eta_L())
        
        return (self.R*self.F/self.T)*( numerator_H + numerator_M + numerator_L )/(denominator_H + denominator_M + denominator_L)
    
    def algebraic_condition(self):
        return self.I - self.i_H() - self.i_M() - self.i_L()
    
    def f(self):
        
        return np.array([self.algebraic_condition(), self.dS8dt(), self.dS4dt(), self.dS2dt(), self.dS1dt(), self.dSpdt()])
    
    def record_data(self):
        self.t_data.append(self.state[0])
        self.V_data.append(self.state[1])
        self.S8_data.append(self.state[2])
        self.S4_data.append(self.state[3])
        self.S2_data.append(self.state[4])
        self.S1_data.append(self.state[5])
        self.Sp_data.append(self.state[6])
        
        
        
        
        
        

In [701]:
model1 = model()

In [702]:
model1.algebraic_condition()

1.297920104725847e-14

In [775]:
class Backward_Euler(model):
    def __init__(self,model):
        model.__init__(self)
    
        # hyper parameters
        self.dt = 0.001
        self.T = 10*3600
        self.V_lower = 1.9
        self.V_upper = 2.6
        
        
    def F_vec(self):
        
        # state should be the latest guess for the next state
        state0 = self.perma_state
        state1 = self.state
        
        f_state1 = self.f()
        
        eqs = [f_state1[0]]
        
        for i in range(1,len(f_state1)):
             eqs.append(state1[i+1]-state0[i+1] - self.dt*f_state1[i])
        
        return np.array(eqs)
    
    def num_deriv_F(self):
        
        state0 = self.state
        F0 = self.F_vec()
        num_deriv_F_array = []
        
        for i in range(1,len(state0)):
            self.state[i] +=  1e-1
            
            # this will add an array at each loop
            num_deriv_F_array.append( ( np.array(self.F_vec()) - np.array(F0)) *10 ) 
            
            self.state = state0
            
            # returns a matrix 
        return np.array(num_deriv_F_array)
    
    def gradient(self):
        
        # evaulate F at with latest state guess
        F1 =  self.F_vec()
        F1_deriv_matrix = self.num_deriv_F()
        
        gradient = []
        for F1_deriv in F1_deriv_matrix:
            gradient.append(2*np.dot(F1,F1_deriv))
        
        return np.array(gradient)
    
    def grad_desc_step(self):
        
        lamb = 1e-70
        
        state0 = self.state
        
        state1 = state0[1:] - lamb*self.gradient()
        
        self.state[1:] = state1
   
    def H(self):
        
        return np.dot(self.F_vec(), self.F_vec())
        
    def grad_desc(self):
        
        tol = 1e-6
        
        while self.H() > tol:
            
            self.grad_desc_step()
            
    def BE_step(self):
        
        self.grad_desc()
        self.perma_state = self.state
        self.perma_state[0] += self.dt
        
        # provide new guess 
        self.state[1:] *= (1 + 1e-12)
        
    def BE_method(self):
        
        while self.t < self.T:
            self.BE_step()

In [819]:
self = Backward_Euler(model)
self.state = self.perma_state

while self.H() > 1e-8:
    F_vec0 = self.F_vec()
    grad = []
    for index in range(1,len(self.state)):

        # change one state variable by a small amount
        self.state[index] += 1e-12
        # reevaluate the numerical method equations
        F_vec1 = self.F_vec()
        # calculuate numerical derivative to numerical ODE equations
        num_deriv = np.array((F_vec1 - F_vec0)/ (1e-12))
        print(num_deriv)
        # reset state
        self.state = self.perma_state
        #store gradient
        grad.append(np.dot(num_deriv,F_vec0))
    
    self.state[1:] = self.state[1:] - (1e-3)*np.array(grad)
    print(grad)

[ 5.51302608e+02 -4.87557790e-04  2.43778895e-04  1.21889150e-04
  1.21891429e-04  0.00000000e+00]
[-1.01671658e+19  2.69747584e+13 -2.69747584e+13  1.21889150e-04
  1.21891429e-04  0.00000000e+00]
[-5.47852911e+17  1.45328639e+12 -1.45316854e+12 -1.17845456e+08
  1.21891429e-04  0.00000000e+00]
[-5.47852911e+17  1.45328639e+12 -1.45316854e+12 -1.17845456e+08
 -1.66696084e-05  0.00000000e+00]
[-5.47852911e+17  1.45328639e+12 -1.45316854e+12 -1.17845456e+08
  9.74653301e-02 -9.42982536e-02]
[-5.47852911e+17  1.45328639e+12 -1.45316854e+12 -1.17845456e+08
  9.76844202e-02 -9.45173437e-02]
[1.1489809284233808e-08, -131961.68845845084, -7109.043807841485, -7109.043807854564, -7109.043789786638, -7109.043789745356]
[ 4.87256330e+05 -9.91838556e-01  8.43457248e-01  1.47347412e-01
  0.00000000e+00  0.00000000e+00]
[ 4.87256330e+05 -9.91838556e-01  8.43457248e-01  1.47347412e-01
  0.00000000e+00  0.00000000e+00]
[ 4.87751095e+05 -9.93684302e-01  8.45573611e-01  1.47083734e-01
  0.00000000e+00 

/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:105: RuntimeWarning: overflow encountered in sinh
  return -2 * self.ih0 * self.ar * np.sinh(self.iH_coef * self.eta_H())
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:109: RuntimeWarning: overflow encountered in sinh
  return -2 * self.im0 * self.ar * np.sinh(self.iM_coef * self.eta_M())
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:113: RuntimeWarning: overflow encountered in sinh
  return -2 * self.il0 * self.ar * np.sinh(self.iL_coef * self.eta_L())
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:133: RuntimeWarning: invalid value encountered in double_scalars
  return  (self.ns8 * self.Ms * self.i_H() / (self.nH * self.F)) + self.k_s() * S8 - (self.ns4 * self.Ms * self.i_M() / (self.nM * self.F))
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:136: RuntimeWarning: invalid value

In [814]:
self.state

array([ 0.00000000e+00,  2.00000000e+00, -1.03528268e+52, -3.03257383e+06,
        1.14879642e-02,  9.99952745e-04,  4.30000000e-01])

In [812]:
self.H()

1.7751404603640628e-08

In [805]:
F_vec1 = self.F_vec()
F_vec1

array([ 1.23701049e-12, -1.08249953e-18,  5.41249767e-19, -1.39952327e-08,
        9.42179426e-05, -9.42039474e-05])

In [806]:
(F_vec1 - F_vec0)/ (self.state[1]*1e-15)

array([ 6.12015646e+02, -5.41249767e-04,  2.70624883e-04,  1.35311859e-04,
        1.35525272e-04,  0.00000000e+00])

In [795]:
self.num_deriv_F()

array([[ 9.55697872e+02, -8.44734720e-04,  4.22367360e-04,
         2.10838318e-04,  2.11529042e-04,  0.00000000e+00],
       [-3.55572026e+08,  9.43378606e+02, -9.43379029e+02,
         2.10838318e-04,  2.11529042e-04,  0.00000000e+00],
       [-1.07635337e+05,  9.03816361e-02,  7.63563788e-03,
        -9.82288031e-02,  2.11529042e-04,  0.00000000e+00],
       [-5.76120990e+04,  9.03816361e-02, -5.89180221e-02,
        -3.15777847e-02,  1.14170696e-04,  0.00000000e+00],
       [-5.59283818e+04,  9.03816361e-02, -5.89180221e-02,
        -3.26945626e-02,  9.55291942e-02, -9.42982456e-02],
       [-5.59283818e+04,  9.03816361e-02, -5.89180221e-02,
        -3.26945626e-02,  1.17678098e-01, -1.16447149e-01]])

In [780]:
self.gradient()

array([-6.71150135e+08, -6.66063440e+08, -6.68410502e+08, -6.60323046e+08,
       -7.09622399e+08, -7.09622399e+08])

In [781]:
self.grad_desc_step()

In [782]:
self.state

array([0.        , 2.3       , 0.3       , 0.3       , 0.31148797,
       0.301     , 0.73      ])

In [765]:
self.state

array([        0.        , -14125205.17961981, -14119875.03653646,
       -14123064.42658241, -12713447.05062995, -15845366.43464592,
       -15845366.00639238])

In [766]:
k1 = np.insert(self.f(),0,[1])
k1

/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:76: RuntimeWarning: invalid value encountered in log
  return self.EH0 + self.E_H_coef * (np.log(self.f_h) + 0.25*np.log(S8) - 0.5*np.log(S4) )
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:81: RuntimeWarning: invalid value encountered in log
  return self.EM0 + self.E_M_coef * (np.log(self.f_m) + 0.5*np.log(S4) - np.log(S2) )
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/3267609732.py:87: RuntimeWarning: invalid value encountered in log
  return self.EL0 + self.E_L_coef * (np.log(self.f_l) + 0.5*np.log(S2) - np.log(S1) )


array([1.00000000e+00,            nan,            nan,            nan,
                  nan,            nan, 5.50604453e+16])

In [394]:
self.state = self.state + k1*self.dt/2

In [395]:
self.state

array([5.00000000e-08, 2.00000000e+00, 2.46791766e-63, 8.42516141e-18,
       1.14879664e-02, 1.00000000e-03, 4.30000000e-01])

In [396]:
self.f()

array([ 7.43234421e+53,  9.87693359e-10, -4.94234539e-10, -1.39962198e-05,
       -4.84506315e-06,  1.88407895e-05])

In [316]:
RK2_model.i_H() + RK2_model.i_M() + RK2_model.i_L()

-0.021101115518319908

In [317]:
RK2_model.i_M()

-3.719831368034817e-07

In [318]:
k2 = np.insert(RK2_model.f(), 0, [1])
k2

array([ 1.00000000e+00,  7.43234421e+53,  9.87693359e-10, -4.94234539e-10,
       -1.39962198e-05, -4.84506315e-06,  1.88407895e-05])

In [319]:
RK2_model.dVdt()

7.432344206519731e+53

In [320]:
RK2_model.state + k2*RK2_model.dt

array([ 1.50000000e-07,  7.43234421e+46,  9.87693359e-17, -4.09982925e-17,
        1.14879664e-02,  9.99999999e-04,  4.30000000e-01])

In [321]:
RK2_model.state - state0

array([ 5.00000000e-08,  2.02597761e-09,  0.00000000e+00,  0.00000000e+00,
       -6.99761429e-13, -2.42277769e-13,  9.42024236e-13])

In [322]:
RK2_model.RK_step()

/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/1264420304.py:75: RuntimeWarning: invalid value encountered in log
  return self.EH0 + self.E_H_coef * (np.log(self.f_h) + 0.25*np.log(S8) - 0.5*np.log(S4) )
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/1264420304.py:80: RuntimeWarning: invalid value encountered in log
  return self.EM0 + self.E_M_coef * (np.log(self.f_m) + 0.5*np.log(S4) - np.log(S2) )
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/1264420304.py:154: RuntimeWarning: overflow encountered in cosh
  numerator_L = self.il0*np.cosh(self.iL_coef * self.eta_L())*( 0.5*(self.dS2dt()/S2) - (self.dS1dt()/S1) )
/var/folders/0m/hp806rv14qz74rjlmvs9nzbm0000gp/T/ipykernel_61150/1264420304.py:158: RuntimeWarning: overflow encountered in cosh
  denominator_L = self.nL*self.il0*np.cosh(self.iL_coef * self.eta_L())


In [323]:
RK2_model.state

array([2.0e-07,     nan,     nan,     nan,     nan,     nan, 4.3e-01])

In [324]:
RK2_model.E_H()

nan

In [325]:
f_values

array([-6.28116235e+01, -2.46791766e-67,  2.46791766e-67,  1.39952327e-05,
       -9.42179426e-02,  9.42039474e-02])

In [326]:
S8 = self.state[2]
S4 = self.state[3]
S2 = self.state[4]
S1 = self.state[5]
numerator_H = -self.ih0*np.cosh(self.iH_coef * self.eta_H())*( (self.dS8dt()/S8) - 2*(self.dS4dt()/S4 ))
numerator_M = -self.im0*np.cosh(self.iM_coef * self.eta_M())*( (self.dS4dt()/S4) - 2*(self.dS2dt()/S2 ))
numerator_L = -self.il0*np.cosh(self.iL_coef * self.eta_L())*( (self.dS2dt()/S2) - 2*(self.dS1dt()/S1) )

denominator_H = self.nH*self.ih0*np.cosh(self.iH_coef * self.eta_H())
denominator_M = self.nM*self.im0*np.cosh(self.iM_coef * self.eta_M())
denominator_L = self.nL*self.il0*np.cosh(self.iL_coef * self.eta_L())



NameError: name 'self' is not defined